In [131]:
import requests
from requests import get
from bs4 import BeautifulSoup
import numpy as np
import csv
import requests

In [194]:
supported_languages = ['de-at', 'en-at', 'de-de', 'en-de', 'es-es', 'en-es', 'en-eu', 'fr-fr', 'en-fr', 'it-it', 'en-it', 'en-us', 'en-gb']
source_page = 'https://support.n26.com'

In [192]:
urls = []
markets = []

# Scrapping of all categories pages urls
for lang in supported_languages:
    url_source = source_page +'/'+ lang 
    page = requests.get(url_source)
    soup = BeautifulSoup(page.content, 'lxml')
    for link in soup.find_all('a', class_=['np', 'nq', 'nr', 'ns', 'nt', 'nu', 'x']):
        link = source_page + link.get('href')
        urls.append(link)
        markets.append(lang)
        

In [201]:
# Class for Q&A questions links on categories pages
class_= 'ba bb bc bd be bf bg bh bj bk bl bm bn bq ca cb cc cd ce g hm jv jw jx jy jz ka nc w'
class_ = class_.replace(' ','.')
count = 0

# Writing of CSV file with all Q&A urls for every supported language 
with open('ulrs_by_market_n26.csv', 'w', newline='') as csvfile:
    fieldnames = ['', 'url', 'market']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for j in range(len(urls)) :
        page = requests.get(urls[j])
        soup = BeautifulSoup(page.content, 'lxml')
        for link in soup.select('a.'+class_):
            writer.writerow({'':count, 'url':source_page+link.get('href'), 'market':markets[j]})
            count += 1


In [249]:

scraped_question = {}

# We open the csv file generated above, repertory of all question URL, question ID and extracted texts 

count = 0
count_FAQ = 0
question_market='fr-fr'
with open('scraped_n26.csv','w', newline='') as writecsv:
    fieldnames = ['ID', 'FAQ_id', 'locale', 'market', 'question', 'answer']
    writer = csv.DictWriter(writecsv, fieldnames=fieldnames)
    writer.writeheader()
    with open('ulrs_by_market_n26.csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['market'] != question_market:
                count_FAQ = 0
            question_url = row['url'] 
            question_market = row['market']
            page = requests.get(question_url)
            soup = BeautifulSoup(page.content, 'lxml')
            question_text = soup.find('div', id="main")
            if question_text == None : 
                print(question_url )
            question = question_text.find('h1').text
            question_text.find('h1').decompose()
            answer = question_text.get_text(separator=' ')
            writer.writerow({'ID':count, 'FAQ_id' : count_FAQ , 'locale' : question_market[-2:], 'market' : question_market, 'question':question , 'answer' :answer})
            count += 1
            count_FAQ += 1
